In [2]:
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

import numpy as np
import cv2
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import LambdaCallback
from keras.callbacks import ModelCheckpoint
import pandas as pd
import keras

In [ ]:
def vgg13_model(n_classes):
    model = Sequential()
    model.add(Conv2D(64, (3, 3), strides=(1, 1), input_shape=(64, 64, 1), padding='same', activation='relu',
                     kernel_initializer='glorot_uniform'))
    model.add(Conv2D(64, (3, 3), strides=(1, 1), padding='same', activation='relu', kernel_initializer='glorot_uniform'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    model.add(Conv2D(128, (3, 3), strides=(1, 1), padding='same', activation='relu', kernel_initializer='glorot_uniform'))
    model.add(Conv2D(128, (3, 3), strides=(1, 1), padding='same', activation='relu', kernel_initializer='glorot_uniform'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    model.add(Conv2D(256, (3, 3), strides=(1, 1), padding='same', activation='relu', kernel_initializer='glorot_uniform'))
    model.add(Conv2D(256, (3, 3), strides=(1, 1), padding='same', activation='relu', kernel_initializer='glorot_uniform'))
    model.add(Conv2D(256, (3, 3), strides=(1, 1), padding='same', activation='relu', kernel_initializer='glorot_uniform'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    model.add(Conv2D(256, (3, 3), strides=(1, 1), padding='same', activation='relu', kernel_initializer='glorot_uniform'))
    model.add(Conv2D(256, (3, 3), strides=(1, 1), padding='same', activation='relu', kernel_initializer='glorot_uniform'))
    model.add(Conv2D(256, (3, 3), strides=(1, 1), padding='same', activation='relu', kernel_initializer='glorot_uniform'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(1024, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1024, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(n_classes, activation='softmax'))
    return model

In [ ]:
def cv2plt(img):
    print(img.shape)
    if(len(img.shape) == 3):
        plt.axis('off') 
        plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB)) 
        plt.show()
    else:
        plt.axis('off') 
        plt.imshow(img, cmap='gray', vmin=0, vmax=255)  
        plt.show()

In [ ]:
def one_hot(data, num_classes):
    return np.squeeze(np.eye(num_classes)[data.reshape(-1)])

In [ ]:
###test


In [ ]:
def load_data(path):
    label_path = path + "label.csv"
    label = pd.read_csv(label_path, header=None)
    temp = label.to_numpy()
    dataset = []
    labels = []
    for i in range(temp.shape[0]):
        cat = np.argmax(temp[i, 2:10], axis=0)
        #print(cat)
        imgfile = path + temp[i][0]
        img = cv2.cvtColor(cv2.imread(imgfile), cv2.COLOR_BGR2GRAY)
        img=cv2.resize(img,(64,64), interpolation=cv2.INTER_CUBIC)
        dataset.append(img)
        labels.append(cat)
        
    dataset=np.array(dataset)
    labels=np.array(labels)
    labels=one_hot(labels, 8)
    return dataset, labels

In [ ]:
emotion_table = {'neutral'  : 0, 
                 'happiness': 1, 
                 'surprise' : 2, 
                 'sadness'  : 3, 
                 'anger'    : 4, 
                 'disgust'  : 5, 
                 'fear'     : 6, 
                 'contempt' : 7}

In [ ]:
path = "./data/FER2013Train/"
train_x, train_y = load_data(path)
path = "./data/FER2013Valid/"
valid_x, valid_y = load_data(path)
path = "./data/FER2013Test/"
test_x, test_y = load_data(path)


In [ ]:

train_x = train_x.reshape(-1, 64, 64, 1)/255.0  #normalize
valid_x = valid_x.reshape(-1, 64, 64, 1)/255.0  #normalize

print(train_x.shape,valid_x.shape)

In [ ]:
#train_x[0,0,0,0].dtype
#import tensorflow as tf
#train_x, valid_x = tf.cast(train_x,dtype=tf.float32),tf.cast(valid_x,dtype=tf.float32)

In [ ]:
print(test_x.shape)
test_x = test_x.reshape(-1, 64, 64, 1)/255.0  # normalize
#test_x = tf.cast(test_x,dtype=tf.float32)
print(test_x.shape)

In [ ]:
print(train_x[0,:,:,0])
cv2plt(train_x[0,:,:,0])
print(valid_x[0,:,:,0])
cv2plt(valid_x[0,:,:,0])

In [ ]:
n_classes = 8 #设定最后的输出层
training_epochs = 20 #设定整体训练数据共训练多少次
batch_size = 32 #设定每次提取多少张图片
model = vgg13_model(n_classes)

In [ ]:
#model.load_weights('./model/model_32-0.69.hdf5')

In [ ]:
# 每隔一个训练循环就用柱状图显示信息
filepath="./model_vgg10_size64_Adagrad/model_{epoch:02d}-{val_accuracy:.2f}.hdf5"
callbacks = [keras.callbacks.TensorBoard(log_dir='./keras_log_vgg10_sgd'.format(model)), ModelCheckpoint(filepath, monitor='val_accuracy',verbose=1, save_best_only=True)]

In [ ]:
from keras.optimizers import SGD, Adam
sgd = keras.optimizers.Adagrad()

model.compile(
          optimizer="sgd",
          metrics=['accuracy'],   #评价指标
          loss='categorical_crossentropy')   #计算损失---分类交叉熵函数，  binary_crossentropy（二分类）
model.summary()
history = model.fit(train_x, train_y, batch_size=batch_size, epochs=training_epochs, validation_data=(valid_x, valid_y),shuffle = True,callbacks=callbacks)

In [ ]:
history = model.fit(train_x, train_y, batch_size=batch_size, epochs=20, validation_data=(valid_x, valid_y),shuffle = True,callbacks=callbacks)

In [ ]:
# 绘制训练 & 验证的准确率值
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

# 绘制训练 & 验证的损失值
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

In [ ]:
test_x.shape
result = np.zeros([8,8],dtype = int)

print(result)
temp = test_x[1,:,:,:].reshape(-1,64,64,1)
prediction = model.predict(temp)
rr = int(np.argmax(prediction))
label = int(np.argmax(test_y[1]))
print(rr)
print(label)
prediction = model.predict(test_x)
for i in range(test_x.shape[0]):
    rr = int(np.argmax(prediction[i]))
    label = int(np.argmax(test_y[i]))
    result[rr][label] += 1
    
print(result)

In [ ]:
model = vgg13_model(8)
model.load_weights('./model_size64_Adagrad/model_40-0.80.hdf5')

In [ ]:

cv2.ocl.setUseOpenCL(False)
emotion_dict = {0: "neutral", 1: "happiness", 2: "surprise", 3: "sadness", 4: "anger", 5: "disgust", 6: "fear", 7: "contempt"}
cap = cv2.VideoCapture(0)
while True:
    # Find haar cascade to draw bounding box around face
    ret, frame = cap.read()
    if not ret:
        break
    facecasc = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = facecasc.detectMultiScale(gray,scaleFactor=1.3, minNeighbors=5)
    frame = gray
    for (x, y, w, h) in faces:
        cv2.rectangle(frame, (x, y-50), (x+w, y+h+10), (255, 0, 0), 2)
        roi_gray = gray[y:y + h, x:x + w]
        cropped_img=cv2.resize(roi_gray,(64,64), interpolation=cv2.INTER_CUBIC)
        input_data=np.array(cropped_img)
        input_data = input_data.reshape(-1, 64, 64, 1)
        prediction = model.predict(input_data)
        #print(prediction)
        maxindex1 = int(np.argmax(prediction))
        promax1 = prediction[0,maxindex1]
        prediction[0,maxindex1] = 0.0
        maxindex2 = int(np.argmax(prediction))
        promax2 = prediction[0,maxindex2]
        #print(maxindex)
        #if(maxindex1 == 0 and promax2 > 0.1):
        #    maxindex = maxindex2
        #else:
        #    maxindex = maxindex1
        #cv2.putText(frame, emotion_dict[maxindex], (x+20, y-60), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        cv2.putText(frame, emotion_dict[maxindex1] + "{:.2f}".format(promax1), (x+20, y-60), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        cv2.putText(frame, emotion_dict[maxindex2] + "{:.2f}".format(promax2), (x+40, y-30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 2, cv2.LINE_AA)

    cv2.imshow('Video', cv2.resize(frame,(704,576),interpolation = cv2.INTER_CUBIC))
    #cv2.imshow('Video', cv2.resize(gray,(704,576),interpolation = cv2.INTER_CUBIC))
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()